## Workload 1

In [52]:
# Import all necessary libraries and setup the environment for matplotlib
from pyspark.sql import SparkSession, Row
from pyspark.ml.feature import PCA
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, Normalizer, VectorAssembler
from pyspark.ml.linalg import *
from pyspark.sql.types import * 
from pyspark.sql.functions import *
import pyspark.sql.functions as f
import numpy as np
import matplotlib.pyplot as plt

In [53]:
def blank_as_null(x):
    return when(col(x) != "", col(x)).otherwise(None)

In [54]:
spark = SparkSession \
    .builder \
    .appName("Assignment 2") \
    .getOrCreate()
tweeets_data = spark.read.option('multiline','true').json('tweets.json')

In [4]:
tweeets_data.show()

+-------------------+--------------------+-------------------+----------+---------------+-------------------+------------------+--------------------+-------------------+--------------------+
|         created_at|           hash_tags|                 id|replyto_id|replyto_user_id|         retweet_id|   retweet_user_id|                text|            user_id|       user_mentions|
+-------------------+--------------------+-------------------+----------+---------------+-------------------+------------------+--------------------+-------------------+--------------------+
|2021-05-05 23:37:51|                null|1390088382659895296|      null|           null|1390027514332991489|            807095|RT @nytimes: Brea...|           17799542| [{807095, [3, 11]}]|
|2021-05-05 23:37:45|[{[9, 18], BREAKI...|1390088354717474822|      null|           null|1390022155350446082|         380648579|RT @AFP: #BREAKIN...|         1166466828|[{380648579, [3, ...|
|2021-05-05 23:37:42|                null|139

In [14]:
tweets_agg = tweeets_data.groupby("user_id").agg(f.concat_ws(" ", f.collect_list(tweeets_data.retweet_id)).alias('agg_retweets'),
                                    f.concat_ws(" ", f.collect_list(tweeets_data.replyto_id)).alias('agg_replies'))

In [27]:
tweets_agg.show()

+---------+--------------------+-------------------+
|  user_id|        agg_retweets|        agg_replies|
+---------+--------------------+-------------------+
| 15466159| 1390027514332991489|               null|
| 19652471| 1390023742194061312|               null|
| 30616018| 1390026843068239874|               null|
| 32947971| 1390027514332991489|               null|
| 33868781|                null|               null|
| 43301934| 1390022155350446082|               null|
| 51797430| 1390023742194061312|               null|
| 54579715| 1390027514332991489|               null|
|106456868| 1390060755995799552|               null|
|109826391| 1390027514332991489|               null|
|111249239| 1390023742194061312|               null|
|147126487|                null|1390043577703563265|
|157101980| 1390056156756914182|               null|
|202170318|13900713418947502...|               null|
|228531805| 1390027514332991489|               null|
|235000057|                null|              

In [22]:
tweets_agg = tweets_agg.withColumn("agg_retweets", blank_as_null("agg_retweets"))
tweets_agg = tweets_agg.withColumn("agg_replies", blank_as_null("agg_replies"))

In [31]:
tweets_processed = tweets_agg.select('user_id',concat_ws(' ','agg_retweets','agg_replies').alias('agg_tweet_respond'))

In [39]:
tf.select(col('user_id'),col('tf')).show(truncate=False)

+---------+------------------------------------------------------------------+
|user_id  |tf                                                                |
+---------+------------------------------------------------------------------+
|15466159 |(262144,[23124],[1.0])                                            |
|19652471 |(262144,[204285],[1.0])                                           |
|30616018 |(262144,[158844],[1.0])                                           |
|32947971 |(262144,[23124],[1.0])                                            |
|33868781 |(262144,[249180],[1.0])                                           |
|43301934 |(262144,[178090],[1.0])                                           |
|51797430 |(262144,[204285],[1.0])                                           |
|54579715 |(262144,[23124],[1.0])                                            |
|106456868|(262144,[95155],[1.0])                                            |
|109826391|(262144,[23124],[1.0])                   

In [51]:

cv_data.select(col('user_id'),col('features')).show(truncate=False)

+---------+--------------------------------------------+
|user_id  |features                                    |
+---------+--------------------------------------------+
|15466159 |(656,[0],[1.0])                             |
|19652471 |(656,[1],[1.0])                             |
|30616018 |(656,[114],[1.0])                           |
|32947971 |(656,[0],[1.0])                             |
|33868781 |(656,[2],[1.0])                             |
|43301934 |(656,[3],[1.0])                             |
|51797430 |(656,[1],[1.0])                             |
|54579715 |(656,[0],[1.0])                             |
|106456868|(656,[14],[1.0])                            |
|109826391|(656,[0],[1.0])                             |
|111249239|(656,[1],[1.0])                             |
|147126487|(656,[480],[1.0])                           |
|157101980|(656,[26],[1.0])                            |
|202170318|(656,[3,16,25,87,100],[1.0,1.0,1.0,1.0,1.0])|
|228531805|(656,[0],[1.0])     

In [41]:
tokenizer = Tokenizer(inputCol='agg_tweet_respond',
    outputCol="vectors")
tweets_vectors = tokenizer.transform(tweets_processed)

In [42]:
hashingTF = HashingTF(inputCol="vectors", outputCol="tf")
tf = hashingTF.transform(tweets_vectors)

In [43]:
selected_id = 202170318
tweets_user_filtered = tf_data.where(f'user_id = {selected_id}')
compare_vector = tweets_user_filtered.first()['tf']

NameError: name 'tf_data' is not defined

In [44]:
compare_vector

NameError: name 'compare_vector' is not defined

In [45]:
def cos_sim(a,b=compare_vector):
    return float(a.dot(b) / (a.norm(2) * b.norm(2)))
cos_function = udf(cos_sim, FloatType())

NameError: name 'compare_vector' is not defined

In [46]:
tf_data = tf_data.withColumn("CosineSim",cos_function('tf'))
tf_data = tf_data.where(f'user_id <> {selected_id}')

NameError: name 'tf_data' is not defined

In [47]:
tf_data.show(truncate=False)

NameError: name 'tf_data' is not defined

In [48]:
sorted_output_tf = tf_data.filter(tf_data.CosineSim > 0).sort(col('CosineSim').desc())
sorted_output_tf.show(5,truncate=False)

NameError: name 'tf_data' is not defined

In [49]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="vectors", outputCol="features")

tweets_cv = cv.fit(tweets_vectors)

cv_data = tweets_cv.transform(tweets_vectors)

In [15]:
selected_id = 202170318
tweets_user_filtered = cv_data.where(f'user_id = {selected_id}')
compare_vector = tweets_user_filtered.first()['features']

In [16]:
def cos_sim(a,b=compare_vector):
    return float(a.dot(b) / (a.norm(2) * b.norm(2)))
cos_function = udf(cos_sim, FloatType())

In [17]:
cv_data = cv_data.withColumn("CosineSim",cos_function('features'))
cv_data = cv_data.where(f'user_id <> {selected_id}')

In [18]:
sorted_output_cv = cv_data.filter(cv_data.CosineSim > 0).sort(col('CosineSim').desc())
sorted_output_cv.show(5,truncate=False)

+----------+---------------------------------------+-------------------+---------------------------------------+---------------+-----------+---------------------------------------+------------------------------------------+----------------------+---------+
|user_id   |agg_tweets                             |agg_retweet_users  |agg_retweets                           |agg_reply_users|agg_replies|agg_tweet_respond                      |vectors                                   |features              |CosineSim|
+----------+---------------------------------------+-------------------+---------------------------------------+---------------+-----------+---------------------------------------+------------------------------------------+----------------------+---------+
|3101168904|1390055316683853824 1390055278486310914|380648579 380648579|1390047008971444231 1390022155350446082|null           |null       |1390047008971444231 1390022155350446082|[1390047008971444231, 1390022155350446082]|(656,[

## Workload 2

In [55]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [57]:
tweeets_data.show(5)

+-------------------+--------------------+-------------------+----------+---------------+-------------------+---------------+--------------------+-------------------+--------------------+
|         created_at|           hash_tags|                 id|replyto_id|replyto_user_id|         retweet_id|retweet_user_id|                text|            user_id|       user_mentions|
+-------------------+--------------------+-------------------+----------+---------------+-------------------+---------------+--------------------+-------------------+--------------------+
|2021-05-05 23:37:51|                null|1390088382659895296|      null|           null|1390027514332991489|         807095|RT @nytimes: Brea...|           17799542| [{807095, [3, 11]}]|
|2021-05-05 23:37:45|[{[9, 18], BREAKI...|1390088354717474822|      null|           null|1390022155350446082|      380648579|RT @AFP: #BREAKIN...|         1166466828|[{380648579, [3, ...|
|2021-05-05 23:37:42|                null|139008834528864666

In [66]:
wl2 = tweeets_data.withColumn("mentioned_users", tweeets_data["user_mentions"].getField('id'))
wl2.show(5)

+-------------------+--------------------+-------------------+----------+---------------+-------------------+---------------+--------------------+-------------------+--------------------+---------------+
|         created_at|           hash_tags|                 id|replyto_id|replyto_user_id|         retweet_id|retweet_user_id|                text|            user_id|       user_mentions|mentioned_users|
+-------------------+--------------------+-------------------+----------+---------------+-------------------+---------------+--------------------+-------------------+--------------------+---------------+
|2021-05-05 23:37:51|                null|1390088382659895296|      null|           null|1390027514332991489|         807095|RT @nytimes: Brea...|           17799542| [{807095, [3, 11]}]|       [807095]|
|2021-05-05 23:37:45|[{[9, 18], BREAKI...|1390088354717474822|      null|           null|1390022155350446082|      380648579|RT @AFP: #BREAKIN...|         1166466828|[{380648579, [3, .

In [67]:
wl2_users = wl2.select(col('user_id'),col('mentioned_users'))

In [70]:
wl2_users.show(5)

+-------------------+---------------+
|            user_id|mentioned_users|
+-------------------+---------------+
|           17799542|         807095|
|         1166466828|      380648579|
|1343606436149022723|      191807697|
| 930226031276982273|       15115280|
| 920858307392192513|         807095|
+-------------------+---------------+
only showing top 5 rows



In [69]:
wl2_users = wl2_users.withColumn("mentioned_users", explode("mentioned_users"))

In [71]:
wl2_users_agg = wl2_users.groupBy(col('user_id'),col('mentioned_users')).count()

In [73]:
wl2_users_agg.orderBy(wl2_users_agg["count"].desc()).show(5)

+-------------------+---------------+-----+
|            user_id|mentioned_users|count|
+-------------------+---------------+-----+
|         3094649957|       24259259|    6|
| 898230537244073986|      281877818|    3|
|         3094649957|     3094649957|    3|
|1286417148848480257|       55060090|    3|
|          138011877|      380648579|    2|
+-------------------+---------------+-----+
only showing top 5 rows



In [74]:
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, IndexToString

stringIndexer_uid = StringIndexer(inputCol="user_id", outputCol="user_id_indexed",stringOrderType="frequencyDesc")
model_uid = stringIndexer_uid.fit(wl2_users_agg)

stringIndexer_mentioned = StringIndexer(inputCol="mentioned_users", outputCol="mentioned_users_indexed",stringOrderType="frequencyDesc")
model_mu = stringIndexer_mentioned.fit(wl2_users_agg)

td = model_uid.transform(wl2_users_agg)
wl2_users_transformed = model_mu.transform(td)

In [154]:
wl2_users_transformed.filter(col('mentioned_users_indexed')==138).show()

+-------------------+---------------+-----+---------------+-----------------------+
|            user_id|mentioned_users|count|user_id_indexed|mentioned_users_indexed|
+-------------------+---------------+-----+---------------+-----------------------+
| 806740569707819008|      281877818|    1|         7401.0|                  138.0|
|          109025128|      281877818|    2|          372.0|                  138.0|
| 898230537244073986|      281877818|    3|          888.0|                  138.0|
|1337396414205341699|      281877818|    1|          466.0|                  138.0|
|1364709585458634757|      281877818|    1|          483.0|                  138.0|
+-------------------+---------------+-----+---------------+-----------------------+



In [79]:
model_recs.show(5,truncate=False)

+---------------+------------------------------------------------------------------------------------------+
|user_id_indexed|recommendations                                                                           |
+---------------+------------------------------------------------------------------------------------------+
|1580           |[{114, 3.5947633}, {35, 3.1534054}, {182, 3.0689082}, {176, 2.9330564}, {191, 2.9279106}] |
|4900           |[{114, 3.9289262}, {85, 3.5334418}, {182, 3.2382052}, {176, 3.1983874}, {191, 2.9853811}] |
|5300           |[{114, 3.9289262}, {85, 3.5334418}, {182, 3.2382052}, {176, 3.1983874}, {191, 2.9853811}] |
|6620           |[{114, 3.9289262}, {85, 3.5334418}, {182, 3.2382052}, {176, 3.1983874}, {191, 2.9853811}] |
|7240           |[{138, 3.0787108}, {187, 2.7853408}, {139, 2.7356298}, {116, 2.7342734}, {124, 2.7342734}]|
+---------------+------------------------------------------------------------------------------------------+
only showing top 5 

In [85]:
als = ALS(maxIter=5, regParam=0.01, userCol="user_id_indexed", itemCol="mentioned_users_indexed", ratingCol="count",
          coldStartStrategy="drop")
model = als.fit(wl2_users_transformed)

In [86]:
model_recs = model.recommendForAllUsers(5)

In [87]:
model_recs = model_recs.withColumn("user_id_indexed", model_recs["user_id_indexed"].cast("double"))

In [88]:
udf1 = f.udf(lambda x : 'ID :'+str(x[0])+' Score:'+str(x[1]))
model_recs.select('index','user_id',f.explode('recommendations').alias('id'),udf1(f.col('id')).alias('UserRecommendations')).show(truncate=False)

AnalysisException: cannot resolve '`index`' given input columns: [id, recommendations, user_id_indexed];
'Project ['index, 'user_id, id#6030, <lambda>(id#6030) AS UserRecommendations#6029]
+- Generate explode(recommendations#6021), false, [id#6030]
   +- Project [cast(user_id_indexed#6020 as double) AS user_id_indexed#6024, recommendations#6021]
      +- Project [id#6016 AS user_id_indexed#6020, cast(recommendations#6017 as array<struct<mentioned_users_indexed:int,rating:float>>) AS recommendations#6021]
         +- Project [key#6010 AS id#6016, TopByKeyAggregator(scala.Tuple3)#6015 AS recommendations#6017]
            +- Aggregate [value#6002], [value#6002 AS key#6010, topbykeyaggregator(org.apache.spark.ml.recommendation.TopByKeyAggregator@644075d8, Some(newInstance(class scala.Tuple3)), Some(class scala.Tuple3), Some(StructType(StructField(_1,IntegerType,false), StructField(_2,IntegerType,false), StructField(_3,FloatType,false))), encodeusingserializer(input[0, java.lang.Object, tru

In [310]:
model_recs.select(['user_id'] + [col("recommendations")[i].alias('Recommended User (with score): '+str(i+1)) for i in range(n)]).show(truncate=False)

+-------------------+--------------------------------+--------------------------------+--------------------------------+---------------------------------+--------------------------------+
|user_id            |Recommended User (with score): 1|Recommended User (with score): 2|Recommended User (with score): 3|Recommended User (with score): 4 |Recommended User (with score): 5|
+-------------------+--------------------------------+--------------------------------+--------------------------------+---------------------------------+--------------------------------+
|1155901127738761216|{36326893, 3.1511617}           |{1003895325025865730, 3.1511617}|{42183750, 3.1511521}           |{1146329871418843136, 3.1511521} |{40851610, 2.1324542}           |
|289239316          |{449027230, 4.5872145}          |{281877818, 4.141928}           |{20759034, 3.994655}            |{1003895325025865730, 3.1396317} |{36326893, 3.1396317}           |
|324681796          |{449027230, 4.5872145}          |{28187

In [89]:
als = ALS(userCol="user_id_indexed", itemCol="mentioned_users_indexed", ratingCol="count",
          coldStartStrategy="drop")
model = als.fit(wl2_users_transformed)

model_recs = model.recommendForAllUsers(5).cache()

uid_labels = model_uid.labels
uid_labels_ = array(*[lit(x) for x in uid_labels])

n = 5
mu_labels = model_mu.labels
mu_labels_ = array(*[lit(x) for x in mu_labels])
recommendations = array(*[struct(
    mu_labels_[col("recommendations")[i]["mentioned_users_indexed"]].alias("userId"),
    col("recommendations")[i]["rating"].alias("rating")
) for i in range(n)])

model_recs = model_recs.withColumn("recommendations", recommendations)\
        .withColumn("user_id", uid_labels_[col("user_id_indexed")])

In [94]:
model_recs.select('user_id' , col("recommendations")).show(5)

+-------------------+--------------------+
|            user_id|     recommendations|
+-------------------+--------------------+
|1155901127738761216|[{24259259, 1.257...|
|          289239316|[{26574283, 0.900...|
|          324681796|[{26574283, 0.900...|
|          574799895|[{26574283, 0.900...|
| 772906387676491776|[{133081348, 0.89...|
+-------------------+--------------------+
only showing top 5 rows



In [96]:
model_recs.select(['user_id'] + [col("recommendations")[i].alias('Recommended User: '+str(i+1)) for i in range(n)]).show(5,truncate=False)

+-------------------+----------------------+--------------------------------+--------------------------------+------------------------+----------------------+
|user_id            |Recommended User: 1   |Recommended User: 2             |Recommended User: 3             |Recommended User: 4     |Recommended User: 5   |
+-------------------+----------------------+--------------------------------+--------------------------------+------------------------+----------------------+
|1155901127738761216|{24259259, 1.2575269} |{346102488, 1.1129419}          |{1244013067790364673, 1.0716459}|{3094649957, 0.91468585}|{807095, 0.9016202}   |
|289239316          |{26574283, 0.90062225}|{1891266440, 0.8982357}         |{185025785, 0.8982352}          |{2791419078, 0.89820886}|{216294504, 0.8981522}|
|324681796          |{26574283, 0.90062225}|{1891266440, 0.8982357}         |{185025785, 0.8982352}          |{2791419078, 0.89820886}|{216294504, 0.8981522}|
|574799895          |{26574283, 0.90062225}|{1

In [84]:
from pyspark.sql.functions import array, col, lit, struct

n = 5
mu_labels = model_mu.labels
mu_labels_ = array(*[lit(x) for x in mu_labels])
recommendations = array(*[struct(
    mu_labels_[col("recommendations")[i]["mentioned_users_indexed"]].alias("userId"),
    col("recommendations")[i]["rating"].alias("rating")
) for i in range(n)])

model_recs = model_recs.withColumn("recommendations", recommendations)
model_recs = model_recs.withColumn("user_id", uid_labels_[col("user_id_indexed")])
model_recs.select(col('user_id'),col('recommendations')).show(truncate=False)

AnalysisException: No such struct field mentioned_users_indexed in userId, rating

In [279]:
model_recs.select(col('user_id'),col('recommendations')).show(truncate=False)

+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|user_id            |recommendations                                                                                                                          |
+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|1155901127738761216|[{36326893, 3.1511617}, {1003895325025865730, 3.1511617}, {42183750, 3.1511521}, {1146329871418843136, 3.1511521}, {40851610, 2.1324542}]|
|289239316          |[{449027230, 4.5872145}, {281877818, 4.141928}, {20759034, 3.994655}, {1003895325025865730, 3.1396317}, {36326893, 3.1396317}]           |
|324681796          |[{449027230, 4.5872145}, {281877818, 4.141928}, {20759034, 3.994655}, {1003895325025865730, 3.1396317}, {36326893, 3.1396317}]           |
|574799895          |[{449027230, 4.5872

In [280]:
model_recs

DataFrame[user_id_indexed: int, recommendations: array<struct<userId:string,rating:float>>, user_id: string, index: bigint]

In [244]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import desc

model_recs = model_recs.withColumn("index", monotonically_increasing_id())
model_recs.orderBy(desc("index")).drop("index").show(20,truncate=False)

+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|user_id_indexed|recommendations                                                                                                                                       |user_id            |
+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|7999           |[{587591389, 2.7744668}, {12133382, 2.6727326}, {15250661, 2.592925}, {65201417, 2.5842016}, {91905327, 2.3440156}]                                   |920257832          |
|6819           |[{17154865, 2.5952804}, {14499829, 1.7732091}, {189868631, 1.7732091}, {14159148, 1.7732091}, {104277809, 1.7579473}]                                 |63335497           |
|89             |[{281877818, 1.2333388}, {114632987141

In [81]:
uid_labels = model_uid.labels

In [82]:
uid_labels_ = array(*[lit(x) for x in uid_labels])

In [220]:

test = model_recs.withColumn("t", uid_labels_[col("user_id_indexed")])
test.show(truncate=False)

+---------------+-----------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|user_id_indexed|recommendations                                                                                                                          |t                  |
+---------------+-----------------------------------------------------------------------------------------------------------------------------------------+-------------------+
|1580           |[{36326893, 3.1511617}, {1003895325025865730, 3.1511617}, {42183750, 3.1511521}, {1146329871418843136, 3.1511521}, {40851610, 2.1324542}]|1155901127738761216|
|4900           |[{449027230, 4.5872145}, {281877818, 4.141928}, {20759034, 3.994655}, {1003895325025865730, 3.1396317}, {36326893, 3.1396317}]           |289239316          |
|5300           |[{449027230, 4.5872145}, {281877818, 4.141928}, {20759034, 3.994655}, {1003895325025865730, 3.1396317},

In [246]:
wl2_users_transformed.filter(col('user_id_indexed') == 6306).show()

+----------+---------------+-----+---------------+-----------------------+
|   user_id|mentioned_users|count|user_id_indexed|mentioned_users_indexed|
+----------+---------------+-----+---------------+-----------------------+
|4857661295|      159955958|    1|         6306.0|                   29.0|
+----------+---------------+-----+---------------+-----------------------+



In [235]:
test.printSchema()

root
 |-- user_id_indexed: integer (nullable = false)
 |-- recommendations: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- userId: string (nullable = true)
 |    |    |-- rating: float (nullable = true)
 |-- t: string (nullable = true)

